## Fetch windows, zscores, annotations 

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [3]:
CONSTRAINT_TOOLS = '/scratch/ucgd/lustre-work/quinlan/u6018199/constraint-tools'
CONSTRAINT_TOOLS_DATA = '/scratch/ucgd/lustre-work/quinlan/data-shared/constraint-tools'

In [4]:
import pandas as pd 

pd.set_option('display.max_columns', 50)

TRAIN_SET_LABEL = 'noncoding'

def get_windows_scores_annotations():
    filename = f'{CONSTRAINT_TOOLS_DATA}/benchmark-genome-wide-predictions/chen-et-al-2022/mchale.kmerSizes.trainSets.noisy.enhancer-exon.positive-vista-enhancers.bed'
    df = pd.read_csv(filename, sep='\t')
    df['delta_sf'] = df['N_observed'] - df[f'N_mean_null_7_{TRAIN_SET_LABEL}']
    df['delta_f'] = df[f'N_mean_null_7_{TRAIN_SET_LABEL}'] - df[f'N_mean_null_3_{TRAIN_SET_LABEL}']
    df['window overlaps positive vista enhancer'] = df['positive-vista-enhancer count'] > 0    
    return df

windows_scores_annotations = get_windows_scores_annotations()
windows_scores_annotations

,chromosome,start,end,N_observed,N_bar_7_noncoding,N_mean_null_7_noncoding,N_variance_null_7_noncoding,N_bar_1_noncoding,N_mean_null_1_noncoding,N_variance_null_1_noncoding,N_bar_1_noncoding_noisy,N_mean_null_1_noncoding_noisy,N_variance_null_1_noncoding_noisy,N_bar_3_noncoding,N_mean_null_3_noncoding,N_variance_null_3_noncoding,N_bar_3_noncoding_noisy,N_mean_null_3_noncoding_noisy,N_variance_null_3_noncoding_noisy,N_bar_5_noncoding,N_mean_null_5_noncoding,N_variance_null_5_noncoding,N_bar_5_noncoding_noisy,N_mean_null_5_noncoding_noisy,N_variance_null_5_noncoding_noisy,enhancer overlap,merged_exon overlap,window overlaps enhancer,window overlaps merged_exon,"window overlaps (enhancer, merged_exon)",new chen zscore,positive-vista-enhancer count,delta_sf,delta_f,window overlaps positive vista enhancer
0,chr1,1432000,1433000,260,-0.372778,265.321432,203.777753,1.590921,236.519249,217.834403,1.676924,235.282294,217.264932,-0.247615,263.528599,203.072610,-0.283079,264.038617,203.540686,-0.339806,264.852332,203.910737,-0.340167,264.858613,204.004160,1000.0,NaN,True,False,"(True, False)",4.299894,0,-5.321432,1.792832,False
1,chr1,1435000,1436000,275,-6.280673,368.627875,222.227572,2.097960,243.596443,224.059444,2.193623,242.210058,223.438246,-7.547130,387.939429,223.937877,-7.609011,389.032484,224.595337,-6.810107,375.796139,219.068492,-6.784165,375.458319,219.269877,1000.0,444.0,True,True,"(True, True)",3.331645,0,-93.627875,-19.311554,False
2,chr1,1449000,1450000,225,0.139977,223.137143,177.111728,0.959620,211.569144,195.888493,1.011359,210.858989,195.501382,0.258359,221.559138,177.372370,0.225283,221.996472,177.748203,0.330975,220.612316,175.743926,0.330023,220.624461,175.781765,NaN,311.0,False,True,"(False, True)",1.817267,0,1.862857,1.578005,False
3,chr1,1450000,1451000,313,2.524948,277.859484,193.691881,5.870332,227.882674,210.237742,5.950235,226.828073,209.731395,2.197534,281.968822,199.400099,2.147648,282.634862,199.905291,2.480911,278.322530,195.376527,2.482902,278.288535,195.445943,NaN,758.0,False,True,"(False, True)",1.365863,0,35.140516,-4.109338,False
4,chr1,1451000,1452000,293,2.951595,252.829443,185.226049,4.803930,223.924244,206.755939,4.877094,222.953222,206.278524,2.367303,260.131776,192.772432,2.332657,260.580640,193.155585,2.725116,255.650992,187.839972,2.724052,255.656435,187.931976,NaN,NaN,False,False,"(False, False)",0.666316,0,40.170557,-7.302333,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786072,chr9,137269000,137270000,210,-3.060479,251.650751,185.210985,-0.657779,219.366052,202.746590,-0.596997,218.491272,202.302491,-3.445370,257.657992,191.337298,-3.474260,258.105694,191.720447,-3.228932,254.283943,188.093965,-3.225423,254.247196,188.190706,1000.0,NaN,True,False,"(True, False)",5.276351,0,-41.650751,-6.007241,False
1786073,chr9,137275000,137276000,296,1.990841,268.077912,196.708291,4.575436,229.442056,211.609361,4.655896,228.354529,211.091617,1.921453,268.932391,198.445131,1.881115,269.472162,198.871703,1.974808,268.272874,197.133316,1.977276,268.231681,197.226096,712.0,NaN,True,False,"(True, False)",2.687348,0,27.922088,-0.854479,False
1786074,chr9,137282000,137283000,211,-1.612483,232.654005,180.337479,-0.329061,215.647526,199.475805,-0.272968,214.851260,199.058885,-1.919711,237.194349,186.184553,-1.941497,237.514975,186.512986,-1.783906,235.171685,183.598811,-1.783008,235.161871,183.634632,572.0,641.0,True,True,"(True, True)",-0.090791,0,-21.654005,-4.540344,False
1786075,chr9,137290000,137291000,224,-0.215971,226.884483,178.380280,0.419311,218.046575,201.585989,0.479479,217.199655,201.151534,-0.682673,233.338243,187.113535,-0.708182,233.697227,187.501923,-0.522518,231.079535,183.571865,-0.521048,231.060300,183.607554,NaN,NaN,False,False,"(False, False)",2.162811,0,-2.884483,-6.453759,False


In [5]:
windows_scores_annotations_coding = windows_scores_annotations[windows_scores_annotations['window overlaps merged_exon'] == True]
windows_scores_annotations_noncoding = windows_scores_annotations[windows_scores_annotations['window overlaps merged_exon'] == False]

In [6]:
import seaborn as sns 
sns.set(font_scale = 1.5)

import matplotlib.pyplot as plt 
plt.rcParams.update({
    'font.size': 20,
    'figure.figsize': (18, 10)
})

import numpy as np 

## Compute $w_{s, exact}$ , $w_{s, approx}$ , and $n_s$ for a subset of windows, and fetch the windows' sequences

In [7]:
import sys
sys.path.append(f'{CONSTRAINT_TOOLS}/utilities')

from collections import defaultdict 

from read_model import read_model 
from windows import compute_flanks
from get_p0s_p1s_p2s_p3s import get_substitution_probability
from kmer import compute_kmers 

def initialize_nested_dictionary(value): 
    return {
        s: {
            f: value for f in compute_kmers(kmer_size=4)
        } for s in compute_kmers(kmer_size=3)
    }

def initialize_dictionary(value): 
    return {s: value for s in compute_kmers(kmer_size=3)}

def read_model_wrapper(kmer_size):
    filename = f'{CONSTRAINT_TOOLS}/dist/model-germline-grch38-Nonly.kmerSize-{kmer_size}.trainSet-{TRAIN_SET_LABEL}.json'
    model = read_model(filename)
    return model

def compute_left_right(): 
    left_flank, right_flank = compute_flanks(3)
    left = 3 - left_flank # middle index of 7mer is 3
    right = 3 + right_flank + 1 # middle index of 7mer is 3
    return left, right

def decompose(seven_mer):
    left, right = compute_left_right()
    s = seven_mer[left:right]
    f = seven_mer[:left] + seven_mer[right:]
    return s, f 

def compute_expected_number_ALTs(model_7): 
    k = initialize_nested_dictionary(value=None)
    for seven_mer in model_7['kmerCounts'].keys(): 
        s, f = decompose(seven_mer)
        p1, p2, p3 = (
            get_substitution_probability(model_7, seven_mer, ALT_multiplicity) 
            for ALT_multiplicity in [1, 2, 3]
        )
        k[s][f] = 1*p1 + 2*p2 + 3*p3
    return k

def compute_marginal_counts(n): 
    n_marginal = initialize_dictionary(value=0)
    for s in n.keys(): 
        n_marginal[s] = sum(n[s].values())
    return n_marginal 

def compute_sf_counts_global(model_7): 
    n = initialize_nested_dictionary(value=0)
    for seven_mer in model_7['kmerCounts'].keys():
        s, f = decompose(seven_mer)
        n[s][f] = model_7['kmerCounts'][seven_mer]['count']
    
    n_marginal = compute_marginal_counts(n)
    
    return n, n_marginal

import pysam 
from tqdm import tqdm 

tqdm.pandas()

from kmer import fetch_kmers 
from pack_unpack import pack

def compute_sf_counts_local(chromosome, start, end, genome): 
    region = pack(chromosome, start, end)
    n = initialize_nested_dictionary(value=0)
    for seven_mer in fetch_kmers(region, genome, kmer_size=7, log=False): 
        s, f = decompose(seven_mer)
        n[s][f] += 1
    
    n_marginal = compute_marginal_counts(n)
    
    return n, n_marginal

def compute_cond_f_dist(n, n_marginal):
    p = initialize_nested_dictionary(value=None)
    for s in n.keys():
        if n_marginal[s] > 0: 
            for f in n[s].keys():
                p[s][f] = n[s][f]/n_marginal[s]    
    return p

def compute_w(expected_number_ALTs, f_cond_dist, n_marginal):
    w = initialize_dictionary(value=None)
    for s in n_marginal.keys(): 
        if n_marginal[s] > 0: 
            w[s] = sum(expected_number_ALTs[s][f] * f_cond_dist[s][f] for f in expected_number_ALTs[s].keys())    
    return w 

def compute_expected_number_ALT(s, model_3): # for sanity check
    p1, p2, p3 = (
        get_substitution_probability(model_3, s, ALT_multiplicity)
        for ALT_multiplicity in [1, 2, 3]
    )
    return 1*p1 + 2*p2 + 3*p3

def sanity_check_1(sub_probs, f_cond_dist_global, n_marginal_global): 
    from kmer import CpG
    # check the formula relating k_s to k_sf, where k is expected number of ALTs
    model_3 = read_model_wrapper(kmer_size=3)
    w_approx = compute_w(sub_probs, f_cond_dist_global, n_marginal_global)
    predictions, actuals = [], []
    for s in sub_probs.keys():
        prediction = w_approx[s] # predicted expected number of ALTs
        actual = compute_expected_number_ALT(s, model_3) # actual expected number of ALTs
        if actual > 0.8: print(s, CpG(s), actual)
        predictions.append(prediction) 
        actuals.append(actual)
    plt.plot(predictions, actuals, 'o')
    plt.plot([0, 1], [0, 1])
    
def compute_n_predicted(w, n_marginal_local): # for sanity check
    n_predicted = 0.0
    for s in n_marginal_local.keys(): 
        if n_marginal_local[s] > 0: 
             n_predicted += n_marginal_local[s]*w[s]
    return n_predicted
    
def sanity_check_2(n_exact, n_approx, w_exact, w_approx, n_marginal_local): 
    # n_exact comes from df[f'N_mean_null_7_{TRAIN_SET_LABEL}']
    # n_approx comes from df[f'N_mean_null_3_{TRAIN_SET_LABEL}']
    n_exact_predicted = compute_n_predicted(w_exact, n_marginal_local)
    print('exact', n_exact, n_exact_predicted)
    n_approx_predicted = compute_n_predicted(w_approx, n_marginal_local)
    print('approx', n_approx, n_approx_predicted) 
    
def add_w_and_nMarginal_row(row, genome, expected_number_ALTs, w_approx): 
    n_local, n_marginal_local = compute_sf_counts_local(
        row['chromosome'], 
        row['start'], 
        row['end'], 
        genome
    )
    f_cond_dist_local = compute_cond_f_dist(n_local, n_marginal_local)
    row['n_marginal_local'] = n_marginal_local
    row['w_exact'] = compute_w(expected_number_ALTs, f_cond_dist_local, n_marginal_local)
    row['w_approx'] = w_approx
    return row

def add_w_and_nMarginal(df, genome_filename): 
    model_7 = read_model_wrapper(kmer_size=7)
    expected_number_ALTs = compute_expected_number_ALTs(model_7)
    n_global, n_marginal_global = compute_sf_counts_global(model_7)
    f_cond_dist_global = compute_cond_f_dist(n_global, n_marginal_global)
    # return sanity_check_1(expected_number_ALTs, f_cond_dist_global, n_marginal_global)
    w_approx = compute_w(expected_number_ALTs, f_cond_dist_global, n_marginal_global)
    
    with pysam.FastaFile(genome_filename) as genome: 
        df = df.progress_apply(
            lambda row: add_w_and_nMarginal_row(
                row, genome, expected_number_ALTs, w_approx
            ), 
            axis=1
        )

    return df

# /uufs/chpc.utah.edu/common/HIPAA/u6018199/.conda/envs/constraint-tools/lib/python3.9/site-packages/pyranges/get_fasta.py
# For more pyranges stuff, see: experiments/germline-model/cpg-islands/cpg-islands-compute-zscores.500%2C600.ipynb

import pyranges as pr

def add_sequence(df, genome_filename):
    columns = ['chromosome', 'start', 'end']
    gr = pr.PyRanges(df[columns].rename(columns = {
        column: column.title()
        for column in columns
    }))
    sequence = pr.get_fasta(gr, genome_filename)
    df['sequence'] = sequence
    return df 

pd.set_option('max_colwidth', 20)

def add_w_and_nMarginal_and_sequence(df):
    genome_filename = '/scratch/ucgd/lustre-work/quinlan/data-shared/constraint-tools/reference/grch38/hg38.analysisSet.fa.gz'    
    df = add_w_and_nMarginal(df, genome_filename)
    df = add_sequence(df, genome_filename)
    return df 

add_w_and_nMarginal_and_sequence(windows_scores_annotations.iloc[:3])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 17.39it/s]


,chromosome,start,end,N_observed,N_bar_7_noncoding,N_mean_null_7_noncoding,N_variance_null_7_noncoding,N_bar_1_noncoding,N_mean_null_1_noncoding,N_variance_null_1_noncoding,N_bar_1_noncoding_noisy,N_mean_null_1_noncoding_noisy,N_variance_null_1_noncoding_noisy,N_bar_3_noncoding,N_mean_null_3_noncoding,N_variance_null_3_noncoding,N_bar_3_noncoding_noisy,N_mean_null_3_noncoding_noisy,N_variance_null_3_noncoding_noisy,N_bar_5_noncoding,N_mean_null_5_noncoding,N_variance_null_5_noncoding,N_bar_5_noncoding_noisy,N_mean_null_5_noncoding_noisy,N_variance_null_5_noncoding_noisy,enhancer overlap,merged_exon overlap,window overlaps enhancer,window overlaps merged_exon,"window overlaps (enhancer, merged_exon)",new chen zscore,positive-vista-enhancer count,delta_sf,delta_f,window overlaps positive vista enhancer,n_marginal_local,w_exact,w_approx,sequence
0,chr1,1432000,1433000,260,-0.372778,265.321432,203.777753,1.590921,236.519249,217.834403,1.676924,235.282294,217.264932,-0.247615,263.528599,203.072610,-0.283079,264.038617,203.540686,-0.339806,264.852332,203.910737,-0.340167,264.858613,204.004160,1000.0,NaN,True,False,"(True, False)",4.299894,0,-5.321432,1.792832,False,"{'AAA': 6, 'AAC'...",{'AAA': 0.088171...,{'AAA': 0.109115...,GTGGGCTGGTCTGAGG...
1,chr1,1435000,1436000,275,-6.280673,368.627875,222.227572,2.097960,243.596443,224.059444,2.193623,242.210058,223.438246,-7.547130,387.939429,223.937877,-7.609011,389.032484,224.595337,-6.810107,375.796139,219.068492,-6.784165,375.458319,219.269877,1000.0,444.0,True,True,"(True, True)",3.331645,0,-93.627875,-19.311554,False,"{'AAA': 1, 'AAC'...",{'AAA': 0.065959...,{'AAA': 0.109115...,AGGCCCAGTGTGGTCC...
2,chr1,1449000,1450000,225,0.139977,223.137143,177.111728,0.959620,211.569144,195.888493,1.011359,210.858989,195.501382,0.258359,221.559138,177.372370,0.225283,221.996472,177.748203,0.330975,220.612316,175.743926,0.330023,220.624461,175.781765,NaN,311.0,False,True,"(False, True)",1.817267,0,1.862857,1.578005,False,"{'AAA': 17, 'AAC...",{'AAA': 0.104229...,{'AAA': 0.109115...,cctcctgaactgcccg...


## [TODO #1.2] Choose most constrained window(s), and plot heat map of w_{s,exact}, heat map of w_{s,approx}, and scatter plot of the first w against the second w by ranging over s

## [TODO #1.3] Cluster windows by delta_w_s = w_exact_s - w_approx_s

1. each window has a vector <delta_w_s1, delta_w_s2, ...> 
2. get 100 windows with low zscores, 100 windows with medium zscores, 100 windows with high zscore 
3. do PCA on all 300 windows  
4. ask whether we find three clusters in pc1-pc2 space!

## [TODO #1.4] [GENERAL] Cluster windows by n_s 

1. each window has a vector <n_s1, n_s2, ....> 
2. get 100 windows with low zscores, 100 windows with medium zscores, 100 windows with high zscore 
3. do PCA on all 300 windows
4. ask whether we find three clusters in pc1-pc2 space!

## [TODO #2] [COOL] Dissect the apparently constrained windows using Nucleotide Transformer to make discoveries about genome biology

https://github.com/instadeepai/nucleotide-transformer

0. map windows to their embedding vectors, and show that windows with low-zscores form one cluster, and those with high zscores form a different cluster (similar to n_s analysis above) 
1. [INCREASE SPATIAL RESOLUTION] Compute attention weights for low-zscore windows using template code in https://github.com/instadeepai/nucleotide-transformer/blob/efc3fa5b6bc57ae8ec2704c792cfb038dd0c689e/examples/inference.ipynb. Do pairs of 100bp regions with high attention also have low SNV density? Does this happen for windows that don’t harbor known enhancers, potentially discovering new enhancers? For windows that harbor enhancers, does the attention scores coincide with the enhancers? [Aggregate attention scores, weighting them by an enhancer matrix, similar to, e.g., Fig 3C, to identify layers and heads that are paying attention to enhancers in a window that contains the enhancer.]
2. Compute predicted effect of SNVs in a window using distance between embeddings of REF and ALT alleles for each site (as done in Fig 4A of nucleotide transformer paper). Average predicted effects over all SNP sites in window. Does average predicted effect correlate with zscore of window? One could also repeat this analysis for all monomorphic sites. (This result would nicely parallel results regarding singleton density.)


## [TODO] Aaron's questions (may help discover new biology)

1. Correlate z-scores of windows with average CpG content (relative to genome-wide average) of windows [compare with https://docs.google.com/presentation/d/1XzvnTeJBGnXG9t2UhkrAEBUdemvSNydwC25zglp5Y14/edit#slide=id.g230fb7471c0_0_150 )
2. If 1. Succeeds, then do: Fraction of noncoding windows that overlap CpG islands vs fraction of extreme-zscore windows that overlaps CpG islands 

Are zscores correlated with Ultraconserved elements? 
Fetch ultra conserved elements from zoonomia paper: https://quinlangroup.slack.com/archives/C449KJT3J/p1683166940410939
* [ Aaron’s earlier remarks on UCEs: https://quinlangroup.slack.com/archives/C01U5FGKAHM/p1671468415354519
* [ Also, bear in mind prior conservation scores: 
	* Supplementary Figure 7 of https://www.nature.com/articles/s41588-018-0062-7 shows that low-SNV-density regions are largely different from high conservation regions; 
	* LOGO paper and Venter paper have GERP data links



## [TODO] compute delta_f, where f = methylation level, for each window, and recompute the heat map in the delta_sf-delta_f plane, where this time delta_sf is replaced by z_chen

Hopefully, this will show that measuring how anomalous the methylation levels are can improve upon the Chen zscore for many surrogates of constraint. 